<a href="https://colab.research.google.com/github/ajohn256/Deep-Learning-With-Pytorch/blob/main/CCNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

In [4]:
train_data = torchvision.datasets.CIFAR10(root="./train",train=True,transform=transform,download=True)
test_data = torchvision.datasets.CIFAR10(root="./test",train=False,transform=transform,download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:13<00:00, 12905930.43it/s]


Extracting ./train/cifar-10-python.tar.gz to ./train


100%|██████████| 170498071/170498071 [00:12<00:00, 13721912.08it/s]


Extracting ./test/cifar-10-python.tar.gz to ./test


In [5]:
image,label = train_data[0]
image.size()
# label

torch.Size([3, 32, 32])

In [6]:
class_names = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [7]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,12,5) #(12,28,28)
        self.pool = nn.MaxPool2d(2,2)#(12,14,14)
        self.conv2 = nn.Conv2d(12,24,5)#(24,10,10)

        self.fc1 = nn.Linear(24*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x





In [8]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [9]:
for epoch in range(30):
    print(f"Training epoch {epoch}...")
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs,labels = data
        optimizer.zero_grad()
        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Loss: {running_loss / len(train_loader):.4f}")


Training epoch 0...
Loss: 2.1975
Training epoch 1...
Loss: 1.7424
Training epoch 2...
Loss: 1.5324
Training epoch 3...
Loss: 1.4183
Training epoch 4...
Loss: 1.3155
Training epoch 5...
Loss: 1.2310
Training epoch 6...
Loss: 1.1571
Training epoch 7...
Loss: 1.0946
Training epoch 8...
Loss: 1.0422
Training epoch 9...
Loss: 0.9963
Training epoch 10...
Loss: 0.9547
Training epoch 11...
Loss: 0.9158
Training epoch 12...
Loss: 0.8827
Training epoch 13...
Loss: 0.8493
Training epoch 14...
Loss: 0.8152
Training epoch 15...
Loss: 0.7842
Training epoch 16...
Loss: 0.7560
Training epoch 17...
Loss: 0.7267
Training epoch 18...
Loss: 0.7014
Training epoch 19...
Loss: 0.6773
Training epoch 20...
Loss: 0.6510
Training epoch 21...
Loss: 0.6289
Training epoch 22...
Loss: 0.6075
Training epoch 23...
Loss: 0.5809
Training epoch 24...
Loss: 0.5614
Training epoch 25...
Loss: 0.5384
Training epoch 26...
Loss: 0.5184
Training epoch 27...
Loss: 0.4995
Training epoch 28...
Loss: 0.4797
Training epoch 29...
Los

In [10]:
torch.save(net.state_dict(),'./trained_net.pth')

In [13]:
net = NeuralNet()
net.load_state_dict(torch.load('./trained_net.pth'))

<ipython-input-13-1dfcef34749e>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('./trained_net.pth'))


<All keys matched successfully>

In [14]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
  for data in test_loader:
    images,labels = data
    outputs = net(images)
    _,predicted = torch.max(outputs,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f"Accuracy: {accuracy}%")


Accuracy: 68.17%


In [15]:
#Making new predictions
new_transform = transforms.Compose(
    [
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

In [20]:
img1 = "dd.jpg"
img2 = "cc.jpg"
img3 = "jj.jpg"
image_paths = [img1,img2,img3]
images = [load_image(img) for img in image_paths]

In [21]:
net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _,predicted = torch.max(output,1)
    print(f"Predicted class: {class_names[predicted.item()]}")

Predicted class: horse
Predicted class: cat
Predicted class: plane
